In [11]:
# ROSINA data
# remove rosina artefact

import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
from kneed import KneeLocator

In [12]:
# filepath
rosinapath = "/Users/dkim/Desktop/work/MSM107_analysis/data/06_ROSINA_new" # path for the ROSINA data
ctdfile = "/Users/dkim/Desktop/work/MSM107_analysis/data/04_CTD-Idronaut/msm107_ctd.xlsx" # path for the CTD data
plotpath = "/Users/dkim/Desktop/work/MSM107_analysis/plots" # path for the plots
ctdpath = "/Users/dkim/Desktop/work/MSM107_analysis/data/04_CTD-Idronaut" # path for the CTD data

# read binned particle data
particle_bin = ["Rosi01/Profile01_BinnedData_220909-0901.txt", "Rosi02/Profile02_BinnedData_220909-1310.txt", "Rosi03/Profile03_BinnedData_220909-1328.txt",
"Rosi04/Profile04_BinnedData_220909-1404.txt", "Rosi05/Profile05_BinnedData_220909-1842.txt", "Rosi06/Profile06_BinnedData_220912-1017.txt",
"Rosi07/Profile07_BinnedData_220910-0846.txt", "Rosi08/Profile08_BinnedData_220911-0857.txt", "Rosi09/Profile09_BinnedData_220911-1547.txt",
"Rosi10/Profile10_BinnedData_220909-1508.txt", "Rosi11/Profile11_BinnedData_220911-1601.txt", "Rosi12/Profile12_BinnedData_220912-1135.txt",
"Rosi13/Profile13_BinnedData_220912-1205.txt", "Rosi14/Profile14_BinnedData_220914-1313.txt", "Rosi15/Profile15_BinnedData_220914-1715.txt",
"Rosi16/Profile16_BinnedData_220915-0824.txt", "Rosi17/Profile17_BinnedData_220915-0924.txt", "Rosi18/Profile18_BinnedData_220915-1235.txt",
"Rosi19/Profile19_BinnedData_220915-1349.txt", "Rosi20/Profile20_BinnedData_220915-1413.txt"
]

# read particle image property data
particle_img = ["Rosi01/Profile01_ProcessingLogFile_220909-0901.txt", "Rosi02/Profile02_ProcessingLogFile_220909-1310.txt", "Rosi03/Profile03_ProcessingLogFile_220909-1328.txt",
"Rosi04/Profile04_ProcessingLogFile_220909-1404.txt", "Rosi05/Profile05_ProcessingLogFile_220909-1842.txt", "Rosi06/Profile06_ProcessingLogFile_220912-1017.txt",
"Rosi07/Profile07_ProcessingLogFile_220910-0846.txt", "Rosi08/Profile08_ProcessingLogFile_220911-0857.txt", "Rosi09/Profile09_ProcessingLogFile_220911-1547.txt",
"Rosi10/Profile10_ProcessingLogFile_220909-1508.txt", "Rosi11/Profile11_ProcessingLogFile_220911-1601.txt", "Rosi12/Profile12_ProcessingLogFile_220912-1135.txt",
"Rosi13/Profile13_ProcessingLogFile_220912-1205.txt", "Rosi14/Profile14_ProcessingLogFile_220914-1313.txt", "Rosi15/Profile15_ProcessingLogFile_220914-1715.txt",
"Rosi16/Profile16_ProcessingLogFile_220915-0824.txt", "Rosi17/Profile17_ProcessingLogFile_220915-0924.txt", "Rosi18/Profile18_ProcessingLogFile_220915-1235.txt",
"Rosi19/Profile19_ProcessingLogFile_220915-1349.txt", "Rosi20/Profile20_ProcessingLogFile_220915-1413.txt"
]

# read CTD data
CTD_station_list = [
"01-Profile01/MSM107-P001-C02.txt", "02-Profile03/MSM107-P003-C01.txt",
"03-Profile04/MSM107-P004-C01.txt", "04-Profile05/MSM107-P005-C01.txt",
"05-Profile06/MSM107-P006-C01.txt", "06-Profile07/MSM107-P007-C03.txt",
"07-Profile08/MSM107-P008-C01.txt", "08-Profile10/MSM107-P010-C01.txt",
"09-Profile11/MSM107-P011-C01.txt", "10-Profile12/MSM107-P012-C03.txt", "11-Profile13/MSM107-P013-C03.txt",
"12-Profile14/MSM107-P014-C01.txt", "13-Profile15/MSM107-P015-C01.txt",
"14-Profile16/MSM107-P016-C01.txt", "15-Profile17/MSM107-P017-C02.txt",
"16-Profile18/MSM107-P018-C01.txt", "17-Profile19/MSM107-P019-C01.txt",
"18-Profile20/MSM107-P020-C02.T"
]

In [14]:
ctddepth = pd.read_excel(ctdfile)
bins = [0.04875, 0.061425, 0.0773955, 0.0975183, 0.122873, 0.15482, 0.195073, 0.245792, 0.309698, 0.39022, 0.491677, 0.619513,
        0.780587, 0.983539, 1.23926, 1.56147, 1.96745, 2.47898, 3.12352, 3.93564, 5]
params = ["ImgNo",  "ActContour",  "NoOfContours",  "EquiDia",  "AspectRatio",  "CumTotArea",  "Area",  "CumTotVol",
  "Volume",  "RectX",  "RectY",  "RectWidth",  "RectHeight",  "MinRectWidth",  "MinRectHeight",  "EdgeContact",  "Perimeter",
    "Circularity",  "Elongation",  "Major",  "Feret",  "Mean",   "StdDev",  "Intden",  "Range",  "Meanpos",  "Cv",  "Sr",  "Median",
      "Modal",  "BckgrdMean",  "BckgrdStdDev" ]
profile2run = ["Profile01", "Profile02", "Profile03", "Profile04", "Profile05", "Profile06",
               "Profile07", "Profile08", "Profile09", "Profile10", "Profile11", "Profile12",
               "Profile13", "Profile14", "Profile15", "Profile16", "Profile17", "Profile18",
               "Profile19", "Profile20"] # profilename

for p in profile2run:
    #ctd_idx = [idx for idx, s in enumerate(CTD_station_list) if p in s][0] # ctd file index
    p_idx = [idx for idx, s in enumerate(particle_bin) if p in s][0] # particle file index
    i_idx = [idx for idx, s in enumerate(particle_img) if p in s][0] # img file index
    profile = particle_bin[p_idx].split(os.sep)[0] # rosina profile number e.g. Rosi01
    bindata = pd.read_csv(os.path.expanduser( os.path.join(rosinapath, particle_bin[i_idx]) ), sep="\s+", skiprows=24, header=0,index_col=False, engine="python")
    imgdata = pd.read_csv(os.path.expanduser( os.path.join(rosinapath, particle_img[i_idx]) ),sep="\s+", skiprows=23,index_col=False, names=params, engine="python")
    #ctddf = pd.read_csv(os.path.expanduser( os.path.join(ctdpath, CTD_station_list[ctd_idx]) ), sep="\t", skiprows=16, header=0, engine="python")
    
    ## particle volume and number data into separate dataframe
    numdf = bindata.iloc[:, 1:22] # particle number df
    numdf.columns = bins
    areadf = bindata.iloc[:, 22:43] # particle area df
    areadf.columns = bins
    voldf = bindata.iloc[:, 43:64]  # particle volumn df
    voldf.columns = bins

    ## add depth information based on the sdepth and edepth from CTD and Imagenumber
    imgno = bindata.iloc[:, 0].to_list() # image no
    sdepth, edepth = ctddepth["sdepth"].loc[ctddepth["profile"] == profile].values[0], ctddepth["edepth"].loc[ctddepth["profile"] == profile].values[0]
    dinterval =  (edepth-sdepth) / len(imgno) # interval between the img profile no
    depth = np.arange(sdepth, edepth, dinterval)[:len(imgno)]
    
    # append depth and imgno info to binned particle data 
    numdf["depth"] = depth
    voldf["depth"] = depth
    areadf["depth"] = depth
    numdf["ImgNo"] = bindata.iloc[:, 0]
    voldf["ImgNo"] = bindata.iloc[:, 0]
    areadf["ImgNo"] = bindata.iloc[:, 0]

    # append depth info to image data
    dimgdf = pd.DataFrame({"depth":depth, "ImgNo":imgno})
    imgdf = imgdata.merge(dimgdf, on="ImgNo", how="left")

    # step 0. define the x and y position of the image
    xposition = imgdf["RectX"]
    yposition = imgdf["RectY"]
    # save a figure of the x and y position in scatter plot with histogram
    scatter_axes = plt.subplot2grid((3, 3), (1, 0), rowspan=2, colspan=2)
    x_hist_axes = plt.subplot2grid((3, 3), (0, 0), colspan=2, sharex=scatter_axes)
    y_hist_axes = plt.subplot2grid((3, 3), (1, 2), rowspan=2, sharey=scatter_axes)
    scatter_axes.scatter(xposition, yposition, color="black", s=0.5, alpha=0.7)
    x_hist_axes.hist(xposition, bins= 50, color="black")
    y_hist_axes.hist(yposition, bins= 50,  orientation='horizontal', color="black")
    scatter_axes.set_xlabel("x position")
    scatter_axes.set_ylabel("y position")
    x_hist_axes.tick_params(axis="both", labelbottom =False, labeltop=False, labelleft=False, labelright=False)
    y_hist_axes.tick_params(axis="both", labelbottom=False, labeltop=False, labelleft=False, labelright=False)
    plt.savefig(os.path.expanduser( os.path.join(plotpath, str(profile+"before_xycoodrdination")) ), dpi=600, facecolor="white")
    plt.close()

    # find frequency of each pixel coordination
    xposcount = xposition.value_counts()[:-1].values # count of each pixel coordination
    xposcord = xposition.value_counts()[:-1].index # pixel coordination of image
    yposcount = yposition.value_counts()[:-1].values # count of each pixel coordination
    yposcord = yposition.value_counts()[:-1].index # pixel coordination of image

    # step 1. find the optimal bin number for the histogram. based on the Feedman-Diaconis <- I will skip this method for now
    # instead, I will set the histogram bin numer to 50 as it is working well so far.
    #bin_num = (max(poscountnorm) - min(poscountnorm))  /( ( 2*stats.iqr(poscountnorm) ) / (len(poscountnorm)**(1/3)) ) <- this is the Feedan-Diaconis function to find the best histgram bin numnbers
    xcounts, xcbins = np.histogram(xposcount, 50) # count is height of the histogram, bins is the bin width. i.g, normalized count
    xcountidx = np.where(xposcount > xcbins[1]) # this is the index of the "xposcount" having high number of counts
    ycounts, ycbins = np.histogram(yposcount, 50) # count is height of the histogram, bins is the bin width. i.g, normalized count
    ycountidx = np.where(yposcount > ycbins[1]) # this is the index of the "yposcount" having high number of counts

    """
    # in this part, I will define the duplicated images.
    # First, I will use the index having high frequent X and Y coordination from the step 1.
    # We assume all the images selected from the step 1 as duplicated images.
    # And then we will examine if those images are really duplicated images or not following the next steps.
    # Second, I will use image parametere "AspectRatio" which returns the the ratio of shortest and longest
    # Third, I will use "Perimeter" and "Eauivalent diameter"
    """
    ####################################
    # workging with X and Y coordination
    IX = imgdf.index[ imgdf["RectX"].isin(xposcord[xcountidx]) == True].tolist() # index of freqent coordination in the original img dataframe
    IY = imgdf.index[ imgdf["RectY"].isin(yposcord[ycountidx]) == True].tolist() # index of freqent coordination in the original img dataframe
    # I will have a look at how many duplicated X and Y coordination is 
    uniqXidx = set(IX) # unique index of duplicated x cord
    uniqYidx = set(IY) # unique index of duplicated y cord
    overlapidx = list(uniqXidx & uniqYidx) # overlapping index
    universalidx = list(uniqXidx | uniqYidx) # universal index i.e., all indexes from x coord and y coord
    print("X and Y cord is overlaping in %" ,float(len(overlapidx)) / len(universalidx) * 100) # it returns how many of duplicated x and y coordinates are positioned in the same row
    # Now, I will use overlapping index to remove the duplicated images

    ####################################
    # I will use the following parameters to identify the duplicated images
    A = imgdf["AspectRatio"][overlapidx] # aspec ratio of img from frequent coordination
    X = imgdf["Perimeter"][overlapidx] / imgdf["EquiDia"][overlapidx] # perimeter / equivalent diameter
    E = imgdf["Elongation"][overlapidx] # elongation
    M = imgdf["Mean"][overlapidx] # gray mean level of the image


    ####################################
    """
    I will use Density Based Spatial Clustering of Applications with Noise (DBSCAN) to detect the
    clusters and noises. The clusters are going to be the duplicated images and noises is going to be non-duplicated images
    to use this clustering method, two parameters should be identified.
    Radius of neighborhoods and minimum number of data points in a give radius neighborhood .
    """


    # 1. find best radius
    df = pd.DataFrame({"M": M, "X":X, "E":E}) # create dataframe of parameters
    minsample = 6
    nbrs = NearestNeighbors(n_neighbors= minsample+1).fit(df) # find the best radius value for DBSCAN
    neigh_dist, neigh_ind = nbrs.kneighbors(df)
    sort_neigh_dist = np.sort(neigh_dist, axis = 0)
    k_dist = sort_neigh_dist[:, 4]
    kneedle = KneeLocator(x= range(1, len(neigh_dist)+1), y = k_dist, S= 1.0,
                          curve="concave", direction="increasing", online=True)
    r = kneedle.knee_y # radius of neighborhood
    cluster = DBSCAN(eps=r, min_samples=minsample).fit(df) # DBSCAN clustering
    #print(set(cluster.labels_))

    # plot and save the result of DBSCAN
    fig = plt.figure()
    ax = plt.axes(projection="3d")
    colors = ["yellow", "red", "purple", "black", "blue", "green", "pink"]
    scatter = ax.scatter(df["M"], df["X"], df["E"], c=cluster.labels_, s=5, alpha=0.4, label=colors[0: len(cluster.labels_)])
    ax.legend(handles=scatter.legend_elements()[0])
    fig.savefig(os.path.join(plotpath, str(profile+"DBSCAN")), dpi=600, facecolor="white")
    plt.close()

    # remove outliers from DBSCAN. Again, Outliers are non duplicated images and clusterd point is duplicated images
    nondupidx = np.where(cluster.labels_ == -1)[0].tolist()
    for index in sorted(nondupidx, reverse=True):
      del overlapidx[index]

    ############################################################################################
    # now get rid of the rows coming from the stuck object from image property dataframe
    original_img_len = len(imgdf.index)
    newimgdf = imgdf.drop(overlapidx, axis=0) # remove the rows from the original dataframe of images properties
    filtered_img_len = len(newimgdf.index)
    print((filtered_img_len/original_img_len)*100, "percentage of images are remained from orginal data" ) # print the percentage of images remained from the original dataframe
    print("total number of images to be removed is", original_img_len - filtered_img_len) # print the number of images removed from the original dataframe
    # save the new dataframe of images properties
    newimgdf.to_csv(os.path.expanduser( os.path.join(rosinapath, particle_img[i_idx].replace(p, str("filtered_"+p))) ), index=False)


    # save figure of the x and y position of the images from the new dataframe
    newxposition = newimgdf["RectX"]
    newyposition = newimgdf["RectY"]
    scatter_axes = plt.subplot2grid((3, 3), (1, 0), rowspan=2, colspan=2)
    x_hist_axes = plt.subplot2grid((3, 3), (0, 0), colspan=2, sharex=scatter_axes)
    y_hist_axes = plt.subplot2grid((3, 3), (1, 2), rowspan=2, sharey=scatter_axes)
    scatter_axes.scatter(newxposition, newyposition, color="black", s=0.5, alpha=0.7)
    x_hist_axes.hist(newxposition, bins= 50, color="black")
    y_hist_axes.hist(newyposition, bins= 50,  orientation='horizontal', color="black")
    scatter_axes.set_xlabel("x position")
    scatter_axes.set_ylabel("y position")
    x_hist_axes.tick_params(axis="both", labelbottom =False, labeltop=False, labelleft=False, labelright=False)
    y_hist_axes.tick_params(axis="both", labelbottom=False, labeltop=False, labelleft=False, labelright=False)
    plt.savefig(os.path.expanduser( os.path.join(plotpath, str(profile+"after_xycoodrdination")) ), dpi=600, facecolor="white")
    plt.close()

    ##################################################################################
    # create figures
    # imgdf: has information of each cropped images and corresponding depth information is appended
    fig, ax = plt.subplots(4,4, figsize=(12, 12), facecolor="white", )
    fig.suptitle(profile) # plot name

    ## particle image properties 
    ax1 = ax[0,0] # circularity distribution from original data
    ax1.scatter(imgdf["Circularity"], imgdf["depth"], s=0.5, alpha=0.3, color="black")
    ax1.set_xlabel("Circularity")
    ax1.set_ylabel("Depth (m)")
    ax1.set_xlim(0, 1.2)
    ax1.invert_yaxis()

    ax7 = ax[0,1] # Circularity distribution from filtered data
    ax7.scatter(newimgdf["Circularity"], newimgdf["depth"], s=0.5, alpha=0.3, color="black")
    ax7.set_xlabel("Circularity")
    ax7.set_xlim(0, 1.2)
    ax7.invert_yaxis()

    ax2 = ax[0,2] # Perimeter distribution from original data
    ax2.scatter(imgdf["Perimeter"], imgdf["depth"], s=0.5, alpha=0.3, color="black")
    ax2.set_xlabel("Perimeter")
    ax2.set_xlim(0, 5)
    ax2.invert_yaxis()

    ax8 = ax[0,3] # Perimeter distribution from filtered data
    ax8.scatter(newimgdf["Perimeter"], newimgdf["depth"], s=0.5, alpha=0.3, color="black")
    ax8.set_xlabel("Perimeter")
    ax8.set_xlim(0, 5)
    ax8.invert_yaxis()

    ax3 = ax[1,0] # Mean distribution from original data
    ax3.scatter(imgdf["Mean"], imgdf["depth"], s=0.5, alpha=0.3, color="black")
    ax3.set_xlabel("Mean Grey Level")
    ax3.set_ylabel("Depth (m)")
    ax3.invert_yaxis()

    ax9 = ax[1,1] # Mean distribution from filtered data
    ax9.scatter(newimgdf["Mean"], newimgdf["depth"], s=0.5, alpha=0.3, color="black")
    ax9.set_xlabel("Mean Grey Level")
    ax9.invert_yaxis()

    ax4 = ax[1,2] # StdDev distribution from original data
    ax4.scatter(imgdf["StdDev"], imgdf["depth"], s=0.5, alpha=0.3, color="black")
    ax4.set_xlabel("StdDev Grey level")
    ax4.invert_yaxis()

    ax10 = ax[1,3] # StdDev distribution from filtered data
    ax10.scatter(newimgdf["StdDev"], newimgdf["depth"], s=0.5, alpha=0.3, color="black")
    ax10.set_xlabel("StdDev Grey level")
    ax10.invert_yaxis()

    ax5 = ax[2,0] # Elongation distribution from original data
    ax5.scatter(imgdf["Elongation"], imgdf["depth"], s=0.5, alpha=0.3, color="black")
    ax5.set_xlabel("Elongation")
    ax5.set_ylabel("Depth (m)")
    ax5.invert_yaxis()

    ax11 = ax[2,1] # Elongation distribution from filtered data
    ax11.scatter(newimgdf["Elongation"], newimgdf["depth"], s=0.5, alpha=0.3, color="black")
    ax11.set_xlabel("Elongation")
    ax11.invert_yaxis()

    ax6 = ax[2,2] # Area distribution from original data
    ax6.scatter(imgdf["Area"], imgdf["depth"], s=0.5, alpha=0.3, color="black")
    ax6.set_xlabel("Area")
    ax6.set_xlim(-0.02, 0.5)
    ax6.invert_yaxis()

    ax12 = ax[2,3] # Area distribution from original data
    ax12.scatter(newimgdf["Area"], newimgdf["depth"], s=0.5, alpha=0.3, color="black")
    ax12.set_xlabel("Area")
    ax12.set_xlim(-0.02, 0.5)
    ax12.invert_yaxis()

    ax13 = ax[3,0] # ESD distribution from filtered data
    ax13.scatter(imgdf["EquiDia"], imgdf["depth"], s=0.5, alpha=0.3, color="black")
    ax13.set_xlabel("ESD")
    ax13.set_ylabel("Depth (m)")
    ax13.invert_yaxis()

    ax14 = ax[3,1] # ESD distribution from original data
    ax14.scatter(newimgdf["EquiDia"], newimgdf["depth"], s=0.5, alpha=0.3, color="black")
    ax14.set_xlabel("ESD")
    ax14.set_ylabel("Depth (m)")
    ax14.invert_yaxis()

    ax15 = ax[3,2] # number of particles distribution from filtered data in each 10m depth bin in histogram
    hist_o = np.histogram(imgdf["depth"], range=(np.nanmin(imgdf["depth"]), np.nanmax(imgdf["depth"])),
                            bins=int(imgdf["depth"].max()/10) + 1)
    ax15.plot(hist_o[0], hist_o[1][:-1], color="black")
    ax15.set_xlabel("Number of particles")
    ax15.set_ylabel("Depth (m)")
    ax15.invert_yaxis()

    ax16 = ax[3,3] # number of particles distribution from original data in each 10m depth bin in histogram
    hist_n = np.histogram(newimgdf["depth"], range=(np.nanmin(newimgdf["depth"]), np.nanmax(newimgdf["depth"])),
                           bins=int(newimgdf["depth"].max()/10) + 1)
    ax16.plot(hist_n[0], hist_n[1][:-1], color="black")
    ax16.set_xlabel("Number of particles")
    ax16.set_ylabel("Depth (m)")
    ax16.invert_yaxis()
    
    fig.tight_layout()
    fig.savefig(os.path.join(plotpath, str(profile+"DBSCAN_filter_img")), dpi=300, facecolor="white")
    plt.close()


    """
    ## CTD data
    ax7 = ax[0,2]
    ax7.scatter(ctddf["Temperature"], ctddf["Depth"], s=0.5, alpha=0.3, color="black")
    ax7.set_xlabel("Temperature")
    #ax7.set_xlim(-0.02, 0.5)
    ax7.invert_yaxis()

    ax8 = ax[0,3]
    ax8.scatter(ctddf["Salinity"], ctddf["Depth"], s=0.5, alpha=0.3, color="black")
    ax8.set_xlabel("Salinity")
    #ax8.set_xlim(-0.02, 0.5)
    ax8.invert_yaxis()

    ax9 = ax[1,2]
    ax9.scatter(ctddf["Optical O2%"], ctddf["Depth"], s=0.5, alpha=0.3, color="black")
    ax9.set_xlabel("Oxygen")
    #ax9.set_xlim(-0.02, 0.5)
    ax9.invert_yaxis()

    ax10 = ax[1,3]
    ax10.scatter(ctddf["Fluorometer AutoScale"], ctddf["Depth"], s=0.5, alpha=0.3, color="black")
    ax10.set_xlabel("Fluorescence")
    #ax10.set_xlim(-0.02, 0.5)
    ax10.invert_yaxis()

    ## particle data
    ax11 = ax[2,2]
    ax11.scatter(numdf.iloc[:, 0:-1].sum(axis=1), numdf["depth"], s=0.5, alpha=0.3, color="black")
    ax11.set_xlabel("total particle nunber (#/L)")
    ax11.set_xlim(0, 300)
    ax11.invert_yaxis()

    ax12 = ax[2,3]
    ax12.scatter(voldf.iloc[:, 0:-1].sum(axis=1), voldf["depth"], s=0.5, alpha=0.3, color="black")
    ax12.set_xlabel("total particle volume ppm")
    ax12.set_xlim(0, 0.5)
    ax12.invert_yaxis()
    """
 

X and Y cord is overlaping in % 94.68864468864469
65.88053780030857 percentage of images are remained from orginal data
total number of images to be removed is 1548
X and Y cord is overlaping in % 57.96812749003985
86.4470588235294 percentage of images are remained from orginal data
total number of images to be removed is 288
X and Y cord is overlaping in % 27.9445727482679
89.29856115107914 percentage of images are remained from orginal data
total number of images to be removed is 119
X and Y cord is overlaping in % 91.01876675603218
63.913392141138736 percentage of images are remained from orginal data
total number of images to be removed is 1350
X and Y cord is overlaping in % 96.13537617196432
52.65891385344925 percentage of images are remained from orginal data
total number of images to be removed is 4193
X and Y cord is overlaping in % 89.62406015037594
71.38728323699422 percentage of images are remained from orginal data
total number of images to be removed is 594
X and Y cord i